<a href="https://colab.research.google.com/github/Omarkouta73/Blue-Book-For-Bulldozers/blob/main/MarkupLM/Fine_tune_MarkupLMForTokenClassification_on_a_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

First, we install 🤗 Transformers.

We also install 🤗 Evaluate and Seqeval, for computing metrics like F1, recall and precision.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install -q evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00


## Prepare dataset

Next, let's load a toy dataset which we'll use to fine-tune MarkupLM on.

The goal for the model is to label nodes of HTML strings with the appropriate class.

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Appgain` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Appgain`


In [4]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path='/content/drive/MyDrive/MarkupLM/Notebooks/Tutorial notebooks/snippet_of_codes/atoydataset',
    path_in_repo="toy_dataset",
    repo_id="nielsr/markuplm-toy-dataset",
    repo_type="dataset",
)

ValueError: Provided path: '/content/drive/MyDrive/MarkupLM/Notebooks/Tutorial notebooks/snippet_of_codes/atoydataset' is not a directory

In [5]:
import os
import json
from huggingface_hub import hf_hub_download

file = hf_hub_download(repo_id="nielsr/markuplm-toy-dataset", filename="label.json", repo_type="dataset")

with open(file) as f:
    labels = json.load(f)

for k,v in labels.items():
  print(k,v)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


label.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

0000 {'model': 'Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE', 'price': '$89.99', 'manufacturer': 'Samsung'}
0001 {'model': 'Sony Cyber-shot 10.1 Megapixel Digital Camera - Silver - DSCS930S', 'price': '$55.99', 'manufacturer': 'Sony'}
0002 {'model': 'Nikon Coolpix 12.0 MegaPixel Compact Digital Camera - Red - L22', 'price': '$74.99', 'manufacturer': 'Nikon'}
0003 {'model': 'Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Red Trim - TL210', 'price': '$94.99', 'manufacturer': 'Samsung'}
0004 {'model': 'Nikon Coolpix S60 10.0 MegaPixel Compact Digital Camera - Red - 26134', 'price': '$99.99', 'manufacturer': 'Nikon'}
0005 {'model': 'Olympus 14 Megapixel Digital Camera - Grey - OLYM FE-4030 GREY REF', 'price': '$78.99', 'manufacturer': 'Olympus'}
0006 {'model': 'Kodak EasyShare 8.2 MegaPixel Digital Camera - Silver - CD14', 'price': '$41.99', 'manufacturer': 'Kodak'}
0007 {'model': 'Olympus 10-MegaPixel Digital Camer

In [ ]:
# import os
# import json

# basic_dir = '/content/drive/MyDrive/MarkupLM/Notebooks/Tutorial notebooks/snippet_of_codes/atoydataset'
# with open(os.path.join(basic_dir, 'label.json')) as f:
#     labels = json.load(f)

# for k,v in labels.items():
#   print(k,v)

We'll use MarkupLMFeatureExtractor to extract the nodes and xpaths from the HTML strings. Next, we label the nodes with the appropriate class (in this case, with "model", "price", "manufacturer" or "other").

In [9]:
from transformers import MarkupLMFeatureExtractor

feature_extractor = MarkupLMFeatureExtractor()

# we have 4 labels
id2label = {0: "model", 1:"price", 2:"manufacturer", 3:"other"}
label2id = {label:id for id, label in id2label.items()}

data = []
for k, v in labels.items():
    file_prefix = k
    annotations = v
    print(annotations)
    html_file_path = hf_hub_download(repo_id="nielsr/markuplm-toy-dataset", filename=f"htmls/{file_prefix}.html", repo_type="dataset")
    with open(html_file_path) as f:
        html_code = f.read()
    encoding = feature_extractor(html_code)
    node_labels = [[]]
    for node_text in encoding['nodes'][0]:
        print(f"node text: {node_text}")
        if node_text == annotations['model']:
            node_labels[0].append(label2id['model'])
        elif node_text == annotations['price']:
            node_labels[0].append(label2id['price'])
        elif node_text == annotations['manufacturer']:
            node_labels[0].append(label2id['manufacturer'])
        else:
            node_labels[0].append(label2id['other'])
        print("-------------------------------------")
    data.append({'nodes': encoding['nodes'],
               'xpaths': encoding['xpaths'],
               'node_labels': node_labels, })
    print("===========================================================================")

{'model': 'Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE', 'price': '$89.99', 'manufacturer': 'Samsung'}
node text: eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE
-------------------------------------
node text: Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE
-------------------------------------
node text: 12.2 Megapixel, 3x Optical Zoom, 3x Digital Zoom, Dual LCD Displays - Back:  2.7" Color LCD and Front: 1.5" Color LCD, Image Stabilization, Image Sensor, SD / SDHC Memory Card Slot - Refurbished / Recertified
-------------------------------------
node text: List Price:
-------------------------------------
node text: $179.00
-------------------------------------
node text: Regular Price:
-------------------------------------
node text: $93.99
-------------------------------------
node text: Price:
--------------------------

In [14]:
data[0]['nodes']

[['eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE',
  'Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE',
  '12.2 Megapixel, 3x Optical Zoom, 3x Digital Zoom, Dual LCD Displays - Back:  2.7" Color LCD and Front: 1.5" Color LCD, Image Stabilization, Image Sensor, SD / SDHC Memory Card Slot - Refurbished / Recertified',
  'List Price:',
  '$179.00',
  'Regular Price:',
  '$93.99',
  'Price:',
  '$89.99',
  'You Save:',
  '$89.01 (49.73%)',
  'QTY:',
  'Availability:',
  'In Stock',
  'eCOST Part #:',
  '58093748',
  'Manufacturer:',
  'Samsung',
  'MFG Part #:',
  'TL205 DARK BLUE',
  'Item Condition:',
  'Recertified/Refurbished']]

In [12]:
for k,v in data[0].items():
  print(k,v)

nodes [['eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE', 'Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE', '12.2 Megapixel, 3x Optical Zoom, 3x Digital Zoom, Dual LCD Displays - Back:  2.7" Color LCD and Front: 1.5" Color LCD, Image Stabilization, Image Sensor, SD / SDHC Memory Card Slot - Refurbished / Recertified', 'List Price:', '$179.00', 'Regular Price:', '$93.99', 'Price:', '$89.99', 'You Save:', '$89.01 (49.73%)', 'QTY:', 'Availability:', 'In Stock', 'eCOST Part #:', '58093748', 'Manufacturer:', 'Samsung', 'MFG Part #:', 'TL205 DARK BLUE', 'Item Condition:', 'Recertified/Refurbished']]
xpaths [['/html/head/title', '/html/body/div[1]/div/div/div/div/div/div/div/h1', '/html/body/h2', '/html/body/div[2]/div[5]/div[1]/table/tr[1]/td/table/tr/td[1]', '/html/body/div[2]/div[5]/div[1]/table/tr[1]/td/table/tr/td[2]/table/tr/td/span/span', '/html/body/div[2]/div[5]/div[1]/tabl

In [13]:
for node, label in zip(data[0]['nodes'][0], data[0]['node_labels'][0]):
  print(node, id2label[label])

eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE other
Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE model
12.2 Megapixel, 3x Optical Zoom, 3x Digital Zoom, Dual LCD Displays - Back:  2.7" Color LCD and Front: 1.5" Color LCD, Image Stabilization, Image Sensor, SD / SDHC Memory Card Slot - Refurbished / Recertified other
List Price: other
$179.00 other
Regular Price: other
$93.99 other
Price: other
$89.99 price
You Save: other
$89.01 (49.73%) other
QTY: other
Availability: other
In Stock other
eCOST Part #: other
58093748 other
Manufacturer: other
Samsung manufacturer
MFG Part #: other
TL205 DARK BLUE other
Item Condition: other
Recertified/Refurbished other


## Create PyTorch Dataset

Next, we'll create a regular PyTorch dataset. Each item of the dataset is an HTML string, encoded using MarkupLMProcessor. Note that we initialize the processor with parse_html = False, as we have already parsed the HTML ourselves and we're providing the nodes, xpaths and node labels.

Note that by default, the processor will only label the first token of a given node and label the remaining tokens with -100. you can change this by setting the `only_label_first_subword` attribute of the processor's tokenizer to `False`.

In [17]:
from torch.utils.data import Dataset

class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=512):
        self.data = data
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx]
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, padding="max_length",
                                  max_length=self.max_length, return_tensors="pt")

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding

In [18]:
from transformers import MarkupLMProcessor

processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base")
processor.parse_html = False

dataset = MarkupLMDataset(data=data, processor=processor, max_length=512)

Let's check an example:

In [19]:
example = dataset[0]
for k,v in example.items():
  print(k,v.shape)

input_ids torch.Size([512])
token_type_ids torch.Size([512])
attention_mask torch.Size([512])
xpath_tags_seq torch.Size([512, 50])
xpath_subs_seq torch.Size([512, 50])
labels torch.Size([512])


Let's decode the input_ids back to text:

In [21]:
processor.decode(example['input_ids'])

'<s>eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUESamsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE12.2 Megapixel, 3x Optical Zoom, 3x Digital Zoom, Dual LCD Displays - Back:  2.7" Color LCD and Front: 1.5" Color LCD, Image Stabilization, Image Sensor, SD / SDHC Memory Card Slot - Refurbished / RecertifiedList Price:$179.00Regular Price:$93.99Price:$89.99You Save:$89.01 (49.73%)QTY:Availability:In StockeCOST Part #:58093748Manufacturer:SamsungMFG Part #:TL205 DARK BLUEItem Condition:Recertified/Refurbished</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

Let's verify the correspondence between input_ids and labels. -100 means that those tokens will be ignored by the loss function, hence these won't contribute to the final loss.

In [23]:
for id, label in zip(example['input_ids'].tolist(), example['labels'].tolist()):
  if label != -100:
    print(processor.decode([id]), id2label[label])
  else:
    print(processor.decode([id]), label)

<s> -100
e other
CO -100
ST -100
. -100
com -100
 - -100
 Samsung -100
 12 -100
. -100
2 -100
 Mega -100
Pixel -100
 Compact -100
 Digital -100
 Camera -100
 w -100
/ -100
 Dual -100
 LCD -100
 Dis -100
plays -100
 - -100
 Dark -100
 Blue -100
 - -100
 TL -100
205 -100
 DARK -100
 BL -100
UE -100
Samsung model
 12 -100
. -100
2 -100
 Mega -100
Pixel -100
 Compact -100
 Digital -100
 Camera -100
 w -100
/ -100
 Dual -100
 LCD -100
 Dis -100
plays -100
 - -100
 Dark -100
 Blue -100
 - -100
 TL -100
205 -100
 DARK -100
 BL -100
UE -100
12 other
. -100
2 -100
 Meg -100
apixel -100
, -100
 3 -100
x -100
 Optical -100
 Zoom -100
, -100
 3 -100
x -100
 Digital -100
 Zoom -100
, -100
 Dual -100
 LCD -100
 Dis -100
plays -100
 - -100
 Back -100
: -100
  -100
 2 -100
. -100
7 -100
" -100
 Color -100
 LCD -100
 and -100
 Front -100
: -100
 1 -100
. -100
5 -100
" -100
 Color -100
 LCD -100
, -100
 Image -100
 St -100
abil -100
ization -100
, -100
 Image -100
 Sensor -100
, -100
 SD -100
 / -100
 S

## Create PyTorch Dataloaders

The next step is to create a PyTorch DataLoader, which allows us to get batches from the dataset.

In [24]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

## Define model

We define the model here, which is a MarkupLM-base Transformer, with a token classifier head on top. The token classifier will have randomly initialized weights, while the base Transformer has pre-trained weights.



In [25]:
from transformers import MarkupLMForTokenClassification

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/810 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/277M [00:00<?, ?B/s]

Some weights of MarkupLMForTokenClassification were not initialized from the model checkpoint at microsoft/markuplm-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also create a label_list, where each tag starts with a B (as seqeval expects the labels to be in IOB format).

In [26]:
label_list = ["B-" + x for x in list(id2label.values())]
label_list

['B-model', 'B-price', 'B-manufacturer', 'B-other']

We also define metric calculations (as we'd like to know the F1 score etc. during training). We'll use 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) for that, which is a library containing many tools for evaluating ML models.

In [27]:
import evaluate

# Metric
metric = evaluate.load("seqeval")

def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

def compute_metrics(return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

## Train

Alright, let's train! Here we're training the model in native PyTorch, but of course you could also opt for things like 🤗 Accelerate, 🤗 Trainer, PyTorch Lightning,...

In [33]:
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

model.train()
for epoch in range(5):  # loop over the dataset multiple times
    for batch in tqdm(dataloader):
        # get the inputs;
        inputs = {k:v.to(device) for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**inputs)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print("Loss:", loss.item())

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        preds, refs = get_labels(predictions, labels)
        metric.add_batch(
            predictions=preds,
            references=refs,
        )

    eval_metric = compute_metrics()
    print(f"Epoch {epoch}:", eval_metric)

  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 1.444706916809082
Loss: 0.7568726539611816
Loss: 0.4671686887741089
Loss: 0.49366891384124756
Loss: 0.5006566047668457
Epoch 0: {'manufacturer_precision': 0.14285714285714285, 'manufacturer_recall': 0.1111111111111111, 'manufacturer_f1': 0.125, 'manufacturer_number': 9, 'model_precision': 0.0, 'model_recall': 0.0, 'model_f1': 0.0, 'model_number': 10, 'other_precision': 0.8688524590163934, 'other_recall': 0.8502673796791443, 'other_f1': 0.8594594594594593, 'other_number': 187, 'price_precision': 0.3333333333333333, 'price_recall': 0.1, 'price_f1': 0.15384615384615383, 'price_number': 10, 'overall_precision': 0.7453703703703703, 'overall_recall': 0.7453703703703703, 'overall_f1': 0.7453703703703705, 'overall_accuracy': 0.7453703703703703}


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.3818977177143097
Loss: 0.3774396777153015
Loss: 0.36770913004875183
Loss: 0.31948474049568176
Loss: 0.2962108552455902
Epoch 1: {'manufacturer_precision': 0.0, 'manufacturer_recall': 0.0, 'manufacturer_f1': 0.0, 'manufacturer_number': 9, 'model_precision': 0.0, 'model_recall': 0.0, 'model_f1': 0.0, 'model_number': 10, 'other_precision': 0.8779342723004695, 'other_recall': 1.0, 'other_f1': 0.935, 'other_number': 187, 'price_precision': 1.0, 'price_recall': 0.3, 'price_f1': 0.4615384615384615, 'price_number': 10, 'overall_precision': 0.8796296296296297, 'overall_recall': 0.8796296296296297, 'overall_f1': 0.8796296296296297, 'overall_accuracy': 0.8796296296296297}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Loss: 0.2005840390920639
Loss: 0.2365441471338272
Loss: 0.18958234786987305
Loss: 0.11477822810411453
Loss: 0.09475714713335037
Epoch 2: {'manufacturer_precision': 1.0, 'manufacturer_recall': 0.5555555555555556, 'manufacturer_f1': 0.7142857142857143, 'manufacturer_number': 9, 'model_precision': 1.0, 'model_recall': 0.5, 'model_f1': 0.6666666666666666, 'model_number': 10, 'other_precision': 0.9257425742574258, 'other_recall': 1.0, 'other_f1': 0.9614395886889461, 'other_number': 187, 'price_precision': 1.0, 'price_recall': 0.4, 'price_f1': 0.5714285714285715, 'price_number': 10, 'overall_precision': 0.9305555555555556, 'overall_recall': 0.9305555555555556, 'overall_f1': 0.9305555555555556, 'overall_accuracy': 0.9305555555555556}


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.09760912507772446
Loss: 0.14369723200798035
Loss: 0.13529950380325317
Loss: 0.09236133098602295
Loss: 0.039134129881858826
Epoch 3: {'manufacturer_precision': 0.9, 'manufacturer_recall': 1.0, 'manufacturer_f1': 0.9473684210526316, 'manufacturer_number': 9, 'model_precision': 1.0, 'model_recall': 1.0, 'model_f1': 1.0, 'model_number': 10, 'other_precision': 0.9945054945054945, 'other_recall': 0.9679144385026738, 'other_f1': 0.9810298102981031, 'other_number': 187, 'price_precision': 0.6428571428571429, 'price_recall': 0.9, 'price_f1': 0.75, 'price_number': 10, 'overall_precision': 0.9675925925925926, 'overall_recall': 0.9675925925925926, 'overall_f1': 0.9675925925925926, 'overall_accuracy': 0.9675925925925926}


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.07078348845243454
Loss: 0.1721067577600479
Loss: 0.11769372224807739
Loss: 0.03762887418270111
Loss: 0.020009059458971024
Epoch 4: {'manufacturer_precision': 0.8888888888888888, 'manufacturer_recall': 0.8888888888888888, 'manufacturer_f1': 0.8888888888888888, 'manufacturer_number': 9, 'model_precision': 1.0, 'model_recall': 1.0, 'model_f1': 1.0, 'model_number': 10, 'other_precision': 0.9736842105263158, 'other_recall': 0.9893048128342246, 'other_f1': 0.9814323607427056, 'other_number': 187, 'price_precision': 0.8571428571428571, 'price_recall': 0.6, 'price_f1': 0.7058823529411764, 'price_number': 10, 'overall_precision': 0.9675925925925926, 'overall_recall': 0.9675925925925926, 'overall_f1': 0.9675925925925926, 'overall_accuracy': 0.9675925925925926}


## Inference

Let's try out the model on a new web page for which we have the nodes and xpaths. Here we'll just use one of our training set.

In [34]:
nodes = data[0]['nodes']
xpaths = data[0]['xpaths']
node_labels = data[0]['node_labels']
print("Nodes:", nodes)
print("Xpaths:", xpaths)

Nodes: [['eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE', 'Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE', '12.2 Megapixel, 3x Optical Zoom, 3x Digital Zoom, Dual LCD Displays - Back:  2.7" Color LCD and Front: 1.5" Color LCD, Image Stabilization, Image Sensor, SD / SDHC Memory Card Slot - Refurbished / Recertified', 'List Price:', '$179.00', 'Regular Price:', '$93.99', 'Price:', '$89.99', 'You Save:', '$89.01 (49.73%)', 'QTY:', 'Availability:', 'In Stock', 'eCOST Part #:', '58093748', 'Manufacturer:', 'Samsung', 'MFG Part #:', 'TL205 DARK BLUE', 'Item Condition:', 'Recertified/Refurbished']]
Xpaths: [['/html/head/title', '/html/body/div[1]/div/div/div/div/div/div/div/h1', '/html/body/h2', '/html/body/div[2]/div[5]/div[1]/table/tr[1]/td/table/tr/td[1]', '/html/body/div[2]/div[5]/div[1]/table/tr[1]/td/table/tr/td[2]/table/tr/td/span/span', '/html/body/div[2]/div[5]/div[1]/ta

We'll prepare the example for the model using the processor. Note that we're passing `return_offsets_mapping=True`, as the offsets allow us to determine which tokens are at the start of a given word at which aren't.

In [35]:
# prepare for model
# note that you don't need to prepare node_labels, we just have them available here so we'll compare to the ground truth
encoding = processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, return_offsets_mapping=True, return_tensors="pt").to(device)
for k,v in encoding.items():
  print(k,v.shape)

input_ids torch.Size([1, 192])
token_type_ids torch.Size([1, 192])
attention_mask torch.Size([1, 192])
offset_mapping torch.Size([1, 192, 2])
xpath_tags_seq torch.Size([1, 192, 50])
xpath_subs_seq torch.Size([1, 192, 50])
labels torch.Size([1, 192])


Let's perform a forward pass:

In [36]:
# we don't need the offset mapping and labels for the forward pass
offset_mapping = encoding.pop("offset_mapping")
labels = encoding.pop("labels")

# forward pass
with torch.no_grad():
  outputs = model(**encoding)

The model outputs logits of shape (batch_size, seq_len, num_labels). We just take the highest logit (score) per token as prediction:

In [39]:
predictions = outputs.logits.argmax(-1)
print(predictions)

tensor([[3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 0, 0, 3, 0,
         0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]],
       device='cuda:0')


The model makes predictions at the token level, however we're only interested in the predicted label for the first token of each node.

This can be achieved by accessing the word_ids (to know whether or not the token is a special token or not) and the offset_mapping (to know whether or not the token is the first of a particular node).

In [40]:
results = {"Node": [], "Predicted": [], "Ground truth": []}

for pred_id, word_id, offset, label_id in zip(predictions[0].tolist(), encoding.word_ids(0), offset_mapping[0].tolist(), labels[0].tolist()):
  if word_id is not None and offset[0] == 0:
    print(f"Node: {nodes[0][word_id]}")
    print(f"Predicted: {id2label[pred_id]}")
    print(f"Ground truth: {id2label[label_id]}")
    print("----------")
    results["Node"].append(nodes[0][word_id])
    results["Predicted"].append(id2label[pred_id])
    results["Ground truth"].append(id2label[label_id])

Node: eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE
Predicted: other
Ground truth: other
----------
Node: Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE
Predicted: model
Ground truth: model
----------
Node: 12.2 Megapixel, 3x Optical Zoom, 3x Digital Zoom, Dual LCD Displays - Back:  2.7" Color LCD and Front: 1.5" Color LCD, Image Stabilization, Image Sensor, SD / SDHC Memory Card Slot - Refurbished / Recertified
Predicted: other
Ground truth: other
----------
Node: List Price:
Predicted: other
Ground truth: other
----------
Node: $179.00
Predicted: other
Ground truth: other
----------
Node: Regular Price:
Predicted: other
Ground truth: other
----------
Node: $93.99
Predicted: other
Ground truth: other
----------
Node: Price:
Predicted: other
Ground truth: other
----------
Node: $89.99
Predicted: price
Ground truth: price
----------
Node: You Save:
Predicted: other
Ground tru

Let's pretty print the results as a Pandas dataframe:

In [46]:
import pandas as pd

pd.DataFrame.from_dict(results).head()

,Node,Predicted,Ground truth
0,eCOST.com - Samsung 12.2 MegaPixel Compact Dig...,other,other
1,Samsung 12.2 MegaPixel Compact Digital Camera ...,model,model
2,"12.2 Megapixel, 3x Optical Zoom, 3x Digital Zo...",other,other
3,List Price:,other,other
4,$179.00,other,other


In [47]:
df = pd.DataFrame.from_dict(results)

In [48]:
df[df['Predicted']!="other"]

,Node,Predicted,Ground truth
1,Samsung 12.2 MegaPixel Compact Digital Camera ...,model,model
8,$89.99,price,price
17,Samsung,manufacturer,manufacturer
